# Import libraries, set options, connect to DB

In [1]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
import mapToPoly
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import Itenerary, Base

passWord = os.environ['my_password']
# This commented out one was how I connected to the remote database
# DATABASE_URI = 'postgres://maxcarey:' + passWord + '@totago.cqfm37jhmjmk.ap-southeast-2.rds.amazonaws.com:5432/totago'
DATABASE_URI = 'postgres+psycopg2://maxcarey:' + passWord + '@localhost:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


True
True


# Read in data as pandas data frame, selecting only certain fields

In [2]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time', 'user_id']

In [3]:
df = pd.read_csv('modified_iten.csv', usecols = fields, dtype={"selectedDestination_id" : "str"})

# Wrange field: destinationIDs

In [4]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)


## Remove the 2 cases where the string says null
## Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

## Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)






# Wrangle field: numItenerariesReturned

In [5]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [6]:
#Convert this field to an integer replacing all NA's with zero
# This gets rid of the trailing zeros
df.selectedDestination_name.fillna("", inplace = True)

# Wrangle Field: departureDate

In [7]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))

# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']

print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
35772
number of rows after removal of anamoulous departureDate cases
35764


# Wrangle Field: distinctID

In [8]:
# Create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)
vc = df.primary_key.value_counts()
unique_keys = df.primary_key.unique()

# Wrangle user_id field

In [9]:
df.dtypes

departureDate                object
distinct_id                  object
numItinerariesReturned        int64
selectedDestination_id        int64
selectedDestination_name     object
startFromLocation            object
time                          int64
user_id                     float64
primary_key                  object
dtype: object

In [10]:
# This gets rid of the railing zeros and all of the nas are just
# blank

df['user_id'] = df['user_id'].fillna(0).astype(np.int64)

#Convert to string to be consistent with other fields in database
df['user_id'] = df['user_id'].astype(str)


# Create a subset of the datle with sample method to test geocode and database entry logic

In [11]:
# Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.tail(1000)

# Output this random sample
sampleDf.head(len(sampleDf))

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,user_id,primary_key
34772,2019-08-18,16c90966d38715-0fc7a6ac04dbb38-6427872-3d10d-1...,1,1146,Mailbox Peak,"47.613858998711265,-122.31868693277117",1565768716,827,16c90966d38715-0fc7a6ac04dbb38-6427872-3d10d-1...
34773,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,146,Quarry Rock,Panpacific hotel,1565771855,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34774,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,146,Quarry Rock,"Canada Place Pier, 100 The Pointe, 999 Canada ...",1565772016,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34775,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,146,Quarry Rock,"Canada Place Pier, 100 The Pointe, 999 Canada ...",1565772140,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34776,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,136,Lighthouse Park,"Pan Pacific Hotel Bar, Vancouver, British Colu...",1565773832,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34777,2019-08-18,16c9111838b4e-09fab60505ceac-7373e61-1fa400-16...,1,122,Capilano Canyon,"Metrotown, V5J 1C8, Burnaby, British Columbia,...",1565776771,0,16c9111838b4e-09fab60505ceac-7373e61-1fa400-16...
34778,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,136,Lighthouse Park,"Pan Pacific Hotel Bar, Vancouver, British Colu...",1565780699,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34779,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,146,Quarry Rock,"Canada Place Pier, 100 The Pointe, 999 Canada ...",1565781738,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34780,2019-08-14,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...,1,136,Lighthouse Park,"Canada Place, 999 Canada Pl, Vancouver, Britis...",1565781975,0,16c90c54e451d1-0c3f7ff58d47518-10073e7b-c0000-...
34781,2019-08-14,16c91661d101bd-09ac8e84e9b3e38-4a5a67-13c680-1...,1,1188,Sandy Ridge Mountain Biking,"Portland, Oregon, United States",1565782393,949,16c91661d101bd-09ac8e84e9b3e38-4a5a67-13c680-1...


## Read in the destination data to allow the possibility to pull the correct names

 


In [12]:
sampleDf.dtypes

departureDate               object
distinct_id                 object
numItinerariesReturned       int64
selectedDestination_id       int64
selectedDestination_name    object
startFromLocation           object
time                         int64
user_id                     object
primary_key                 object
dtype: object

# Loop through the rows in the dataframe, geocode, add entry to database

In [13]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(Itenerary.distinctkey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # Try to run the geocode function that returns a dictionary of information
        try:
            geocodeInfo = geocode(locationToGeocode)
            # If geocoding works, set valid to tre
            valid = True

        # If the geocode function doesn't work set valid to false
        except:
            valid = False 

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to none
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            
            zipCodeInfo = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'postal')
            print(zipCodeInfo)
            
            if zipCodeInfo:
                zipCodeMapped = zipCodeInfo[0]
                region = zipCodeInfo[1]
            else:
                zipCodeMapped = 'outsideRegion'
                zipCodeMapped = 'outsideRegion'
            
            barrioInfo = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'barrio')
            
            if barrioInfo:
                barrioMapped = barrioInfo[0]
                region = barrioInfo[1]
            else:
                barrioMapped = 'outsideRegion'
                region = 'outsideRegion'
            
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            
            #if not selectedDestinationName:
            #    
            #    key = str(row["selectedDestination_id"])
            #    
            #    if key in destinations:
            #
            #        # Pull the data out from the dictionary that was created in the cell above
            #        newName = destinations[str(row["selectedDestination_id"])]['name']
        #
            #        # Add the new name to the new row
            #        selectedDestinationName  = newName
            #
            #    # In the case that there is destination that corresponds mark
            #    else:
            #    
            #        # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
            #        selectedDestinationName = "DELETED"
            #        # And overwrite valid to false at this point because there is no destination
            #        valid = False
            
            databaseEntry = Itenerary(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=selectedDestinationName,
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalcode=geocodeInfo['postalCode'],
                                      postalcodemapped=zipCodeMapped,
                                      barriomapped=barrioMapped,
                                      userid=row["user_id"],
                                      region=region,
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = Itenerary(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=row["selectedDestination_name"],
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      userid=row["user_id"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
Entry already inside database
{'formatted_address': '999 Canada Pl, Vancouver, BC V6C 3T4, Canada', 'lat': 49.2888248, 'lng': -123.1111209, 'postalCode': 'V6C 3T4'}
['V6C', 'canada']
{'formatted_address': '999 Canada Pl, Vancouver, BC V6C 3T4, Canada', 'lat': 49.2888248, 'lng': -123.1111209, 'postalCode': 'V6C 3T4'}
['V6C', 'canada']
{'formatted_address': 'Portland, OR, USA', 'lat': 45.5051064, 'lng': -122.6750261}
None
{'formatted_address': 'Portland, OR, USA', 'lat': 45.5051064, 'lng': -122.6750261}
None
{'formatted_address': '3675 W 7th Ave, Vancouver, BC V6R 1W5, Canada', 'lat': 49.2663039, 'lng': -123.1853944, 'postalCode': 'V6R 1W5'}
['V6R', 'canada']
{'formatted_address': '3675 W 7th Ave, Vancouver, BC V6R 1W5, Canada', 'lat': 49.2663039, 'lng': -123.1853944, 'postalCode': 'V6R 1W5'}
['V6R', 'canada']


['V5C', 'canada']
{'formatted_address': '470 Expo Blvd, Vancouver, BC V6B, Canada', 'lat': 49.2781931, 'lng': -123.107241, 'postalCode': 'V6B'}
['V6B', 'canada']
{'formatted_address': '2201 192nd St SE, Bothell, WA 98012, USA', 'lat': 47.825398, 'lng': -122.20303, 'postalCode': '98012'}
['98012', 'washington']
{'formatted_address': '2201 192nd St SE, Bothell, WA 98012, USA', 'lat': 47.825398, 'lng': -122.20303, 'postalCode': '98012'}
['98012', 'washington']
{'formatted_address': '2201 192nd St SE, Bothell, WA 98012, USA', 'lat': 47.825398, 'lng': -122.20303, 'postalCode': '98012'}
['98012', 'washington']
{'formatted_address': '2201 192nd St SE, Bothell, WA 98012, USA', 'lat': 47.825398, 'lng': -122.20303, 'postalCode': '98012'}
['98012', 'washington']
{'formatted_address': '2201 192nd St SE, Bothell, WA 98012, USA', 'lat': 47.825398, 'lng': -122.20303, 'postalCode': '98012'}
['98012', 'washington']
{'formatted_address': '2201 192nd St SE, Bothell, WA 98012, USA', 'lat': 47.825398, 'lng

['V6B', 'canada']
{'formatted_address': '505 W Pender St, Vancouver, BC V6B 6H6, Canada', 'lat': 49.2835563, 'lng': -123.1128655, 'postalCode': 'V6B 6H6'}
['V6B', 'canada']
{'formatted_address': '505 W Pender St, Vancouver, BC V6B 6H6, Canada', 'lat': 49.2835563, 'lng': -123.1128655, 'postalCode': 'V6B 6H6'}
['V6B', 'canada']
{'formatted_address': 'Capilano Suspension Bridge, 3735 Capilano Rd, North Vancouver, BC V7R 4J1, Canada', 'lat': 49.3428609, 'lng': -123.1149244, 'postalCode': 'V7R 4J1'}
['V7S', 'canada']
{'formatted_address': 'Quarry Rock, Baden Powell Trail, North Vancouver, BC V7G 1V6, Canada', 'lat': 49.3334595, 'lng': -122.9364088, 'postalCode': 'V7G 1V6'}
['V7G', 'canada']
{'formatted_address': '305 W Cordova St, Vancouver, BC V6B 1E5, Canada', 'lat': 49.2838058, 'lng': -123.1093501, 'postalCode': 'V6B 1E5'}
['V6B', 'canada']
{'formatted_address': '517 Newton St, Seattle, WA 98109, USA', 'lat': 47.6364064, 'lng': -122.3464275, 'postalCode': '98109'}
['98109', 'washington']

{'formatted_address': '3616 Dayton Ave N B, Seattle, WA 98103, USA', 'lat': 47.6523866, 'lng': -122.3521229, 'postalCode': '98103'}
['98103', 'washington']
{'formatted_address': 'Seattle, WA 98102, USA', 'lat': 47.6338217, 'lng': -122.3215448, 'postalCode': '98102'}
['98102', 'washington']
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng': -122.3320708}
['98164', 'washington']
{'formatted_address': '633 4th Ave W, Seattle, WA 98119, USA', 'lat': 47.6257032, 'lng': -122.3622785, 'postalCode': '98119'}
['98119', 'washington']
{'formatted_address': '15440 NE 13th Pl, Bellevue, WA 98007, USA', 'lat': 47.6217216, 'lng': -122.1335949, 'postalCode': '98007'}
['98007', 'washington']
{'formatted_address': 'Sammamish, WA, USA', 'lat': 47.61626829999999, 'lng': -122.0355736}
['98074', 'washington']
{'formatted_address': '7197 Ash Cres, Vancouver, BC V6P 1S2, Canada', 'lat': 49.2193305, 'lng': -123.1208942, 'postalCode': 'V6P 1S2'}
['V6P', 'canada']
{'formatted_address': 'w Cordov

{'formatted_address': 'Seattle, WA 98121, USA', 'lat': 47.6152474, 'lng': -122.3497446, 'postalCode': '98121'}
['98121', 'washington']
{'formatted_address': '2128 E 53rd Ave, Vancouver, BC V5P 1X7, Canada', 'lat': 49.2210001, 'lng': -123.0639201, 'postalCode': 'V5P 1X7'}
['V5P', 'canada']
{'formatted_address': '112 Stewart St, Seattle, WA 98101, USA', 'lat': 47.61083989999999, 'lng': -122.3408456, 'postalCode': '98101'}
['98101', 'washington']
{'formatted_address': '112 Stewart St, Seattle, WA 98101, USA', 'lat': 47.61083989999999, 'lng': -122.3408456, 'postalCode': '98101'}
['98101', 'washington']
{'formatted_address': '112 Stewart St, Seattle, WA 98101, USA', 'lat': 47.61083989999999, 'lng': -122.3408456, 'postalCode': '98101'}
['98101', 'washington']
{'formatted_address': 'Burrard Station, Vancouver, BC V6C 2Z7, Canada', 'lat': 49.2852945, 'lng': -123.119563, 'postalCode': 'V6C 2Z7'}
['V6C', 'canada']
{'formatted_address': '100 NE Lynnwood Dr, Belfair, WA 98528, USA', 'lat': 47.4279

...

['V6C', 'canada']
{'formatted_address': 'Burrard Station, Vancouver, BC V6C 2Z7, Canada', 'lat': 49.2852945, 'lng': -123.119563, 'postalCode': 'V6C 2Z7'}
['V6C', 'canada']
{'formatted_address': 'Burrard Station, Vancouver, BC V6C 2Z7, Canada', 'lat': 49.2852945, 'lng': -123.119563, 'postalCode': 'V6C 2Z7'}
['V6C', 'canada']
{'formatted_address': 'Burrard Station, Vancouver, BC V6C 2Z7, Canada', 'lat': 49.2852945, 'lng': -123.119563, 'postalCode': 'V6C 2Z7'}
['V6C', 'canada']
{'formatted_address': 'Burrard Station, Vancouver, BC V6C 2Z7, Canada', 'lat': 49.2852945, 'lng': -123.119563, 'postalCode': 'V6C 2Z7'}
['V6C', 'canada']
{'formatted_address': 'Everett, WA, USA', 'lat': 47.9789848, 'lng': -122.2020795}
['98201', 'washington']
{'formatted_address': 'Everett, WA, USA', 'lat': 47.9789848, 'lng': -122.2020795}
['98201', 'washington']
{'formatted_address': '603 3rd Ave W, Seattle, WA 98119, USA', 'lat': 47.6247835, 'lng': -122.3609193, 'postalCode': '98119'}
['98119', 'washington']
{'fo

{'formatted_address': '202 23rd Ave E, Seattle, WA 98112, USA', 'lat': 47.6203565, 'lng': -122.3022261, 'postalCode': '98112'}
['98112', 'washington']
{'formatted_address': '202 23rd Ave E, Seattle, WA 98112, USA', 'lat': 47.6203565, 'lng': -122.3022261, 'postalCode': '98112'}
['98112', 'washington']
{'formatted_address': '310 12th Ave E, Seattle, WA 98102, USA', 'lat': 47.6216815, 'lng': -122.3165095, 'postalCode': '98102'}
['98102', 'washington']
{'formatted_address': 'Cal Anderson Park Reflecting Pool, 1635 11th Ave, Seattle, WA 98122, USA', 'lat': 47.6172908, 'lng': -122.3194892, 'postalCode': '98122'}
['98122', 'washington']
{'formatted_address': 'Cal Anderson Park Reflecting Pool, 1635 11th Ave, Seattle, WA 98122, USA', 'lat': 47.6172908, 'lng': -122.3194892, 'postalCode': '98122'}
['98122', 'washington']
{'formatted_address': '1990 W 41st Ave, Vancouver, BC V6M 1Y4, Canada', 'lat': 49.2344922, 'lng': -123.151786, 'postalCode': 'V6M 1Y4'}
['V6M', 'canada']
{'formatted_address': '

{'formatted_address': '1012 NE 42nd St, Seattle, WA 98105, USA', 'lat': 47.6584948, 'lng': -122.3170101, 'postalCode': '98105'}
['98105', 'washington']
{'formatted_address': '1012 NE 42nd St, Seattle, WA 98105, USA', 'lat': 47.6584948, 'lng': -122.3170101, 'postalCode': '98105'}
['98105', 'washington']
{'formatted_address': '1012 NE 42nd St, Seattle, WA 98105, USA', 'lat': 47.6584948, 'lng': -122.3170101, 'postalCode': '98105'}
['98105', 'washington']
{'formatted_address': '2201 Holdom Ave, Burnaby, BC V5B 3A4, Canada', 'lat': 49.2643868, 'lng': -122.9818363, 'postalCode': 'V5B 3A4'}
['V5B', 'canada']
{'formatted_address': '2201 Holdom Ave, Burnaby, BC V5B 3A4, Canada', 'lat': 49.2643868, 'lng': -122.9818363, 'postalCode': 'V5B 3A4'}
['V5B', 'canada']
{'formatted_address': '2201 Holdom Ave, Burnaby, BC V5B 3A4, Canada', 'lat': 49.2643868, 'lng': -122.9818363, 'postalCode': 'V5B 3A4'}
['V5B', 'canada']
{'formatted_address': 'Quarry Rock, Baden Powell Trail, North Vancouver, BC V7G 1V6, 

['V3H', 'canada']
{'formatted_address': '4435 Slocan St, Vancouver, BC V5R 1Z6, Canada', 'lat': 49.2448506, 'lng': -123.0500652, 'postalCode': 'V5R 1Z6'}
['V5R', 'canada']
{'formatted_address': '4435 Slocan St, Vancouver, BC V5R 1Z6, Canada', 'lat': 49.2448506, 'lng': -123.0500652, 'postalCode': 'V5R 1Z6'}
['V5R', 'canada']
{'formatted_address': '2672 Cambridge St, Vancouver, BC V5K 1L5, Canada', 'lat': 49.2863878, 'lng': -123.0501733, 'postalCode': 'V5K 1L5'}
['V5K', 'canada']
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
['V6C', 'canada']
{'formatted_address': '3843 SW Marine Dr, Vancouver, BC V6N 3Z7, Canada', 'lat': 49.23240939999999, 'lng': -123.1894639, 'postalCode': 'V6N 3Z7'}
['V6N', 'canada']
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
['V6C', 'canada']
{'formatted_address': 'Eastgate Park and Ride, Bellevue, WA 98007, USA', 'lat': 47.5804416, 'lng': -122.1522951, 'postalCode': '98007'}
['98007

{'formatted_address': '1451 Highlands Dr NE, Issaquah, WA 98029, USA', 'lat': 47.5413085, 'lng': -122.0197404, 'postalCode': '98029'}
['98027', 'washington']
{'formatted_address': '1451 Highlands Dr NE, Issaquah, WA 98029, USA', 'lat': 47.5413085, 'lng': -122.0197404, 'postalCode': '98029'}
['98027', 'washington']
{'formatted_address': '3219 Franklin Ave E, Seattle, WA 98102, USA', 'lat': 47.6504475, 'lng': -122.3211605, 'postalCode': '98102'}
['98102', 'washington']
{'formatted_address': '3219 Franklin Ave E, Seattle, WA 98102, USA', 'lat': 47.6504475, 'lng': -122.3211605, 'postalCode': '98102'}
['98102', 'washington']
{'formatted_address': 'Edmonds Station, Burnaby, BC V3N 4X8, Canada', 'lat': 49.2122409, 'lng': -122.9591911, 'postalCode': 'V3N 4X8'}
['V5E', 'canada']
{'formatted_address': 'Burnaby, BC V3N 1T7, Canada', 'lat': 49.2134821, 'lng': -122.9507472, 'postalCode': 'V3N 1T7'}
['V3N', 'canada']
{'formatted_address': '805 146th Ave SE, Bellevue, WA 98007, USA', 'lat': 47.602399